In [1]:
%%bash
wget http://eddylab.org/software/hmmer/hmmer-3.4.tar.gz
tar xf hmmer-3.4.tar.gz
cd hmmer-3.4

--2024-06-16 17:50:20--  http://eddylab.org/software/hmmer/hmmer-3.4.tar.gz
Resolving eddylab.org (eddylab.org)... 96.126.110.11, 2600:3c03::f03c:91ff:fec8:383c
Connecting to eddylab.org (eddylab.org)|96.126.110.11|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 19669667 (19M) [application/x-gzip]
Saving to: ‘hmmer-3.4.tar.gz.1’

     0K .......... .......... .......... .......... ..........  0%  201K 95s
    50K .......... .......... .......... .......... ..........  0%  402K 71s
   100K .......... .......... .......... .......... ..........  0%  162M 47s
   150K .......... .......... .......... .......... ..........  1%  402K 47s
   200K .......... .......... .......... .......... ..........  1%  159M 38s
   250K .......... .......... .......... .......... ..........  1%  173M 31s
   300K .......... .......... .......... .......... ..........  1%  194M 27s
   350K .......... .......... .......... .......... ..........  2%  199M 23s
   400K .......... ........

In [4]:
%%bash
cd hmmer-3.4
./configure
make
make install

configure: Configuring HMMER3 for your system.
checking build system type... x86_64-pc-linux-gnu
checking host system type... x86_64-pc-linux-gnu
checking whether to compile using MPI... no
checking for gcc... gcc
checking whether the C compiler works... yes
checking for C compiler default output file name... a.out
checking for suffix of executables... 
checking whether we are cross compiling... no
checking for suffix of object files... o
checking whether the compiler supports GNU C... yes
checking whether gcc accepts -g... yes
checking for gcc option to enable C11 features... none needed
checking for gcc... (cached) gcc
checking whether the compiler supports GNU C... (cached) yes
checking whether gcc accepts -g... (cached) yes
checking for gcc option to enable C11 features... (cached) none needed
checking how to run the C preprocessor... gcc -E
checking for a BSD-compatible install... /bin/install -c
checking for strip... strip
checking for ranlib... ranlib
checking for ar... /bin/ar


rm: cannot remove 'src/impl': Is a directory
cp: -r not specified; omitting directory 'src/impl_sse'
config.status: error: cannot link or copy src/impl_sse to src/impl


     SUBDIR easel
     SUBDIR miniapps
     SUBDIR libdivsufsort
     SUBDIR src
     SUBDIR impl_sse
     SUBDIR profmark
     SUBDIR easel
     SUBDIR miniapps
     SUBDIR libdivsufsort
     SUBDIR src
     SUBDIR impl_sse
     SUBDIR profmark
if [ ! -d /usr/local/bin ]; then /bin/install -c -d /usr/local/bin; fi
if [ ! -d /usr/local/share/man/man1 ]; then /bin/install -c -d /usr/local/share/man/man1; fi


/bin/install: cannot change permissions of ‘/usr/local/share/man/man1’: No such file or directory
make: *** [Makefile:135: install] Error 1


In [6]:
!wget http://ftp.ebi.ac.uk/pub/databases/Pfam/releases/Pfam35.0/Pfam-A.hmm.gz

--2024-06-16 18:00:31--  http://ftp.ebi.ac.uk/pub/databases/Pfam/releases/Pfam35.0/Pfam-A.hmm.gz
Resolving ftp.ebi.ac.uk (ftp.ebi.ac.uk)... 193.62.193.165
Connecting to ftp.ebi.ac.uk (ftp.ebi.ac.uk)|193.62.193.165|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 293000230 (279M) [application/x-gzip]
Saving to: ‘Pfam-A.hmm.gz’

Pfam-A.hmm.gz       100%[===================>] 279.43M  1.08MB/s    in 4m 5s   

2024-06-16 18:04:36 (1.14 MB/s) - ‘Pfam-A.hmm.gz’ saved [293000230/293000230]



In [7]:
!gunzip Pfam-A.hmm.gz

In [8]:
!hmmpress Pfam-A.hmm

Working...    done.
Pressed and indexed 19632 HMMs (19632 names and 19632 accessions).
Models pressed into binary file:   Pfam-A.hmm.h3m
SSI index for binary model file:   Pfam-A.hmm.h3i
Profiles (MSV part) pressed into:  Pfam-A.hmm.h3f
Profiles (remainder) pressed into: Pfam-A.hmm.h3p


In [17]:
!hmmfetch --index Pfam-A.hmm.h3m

Working...    done.
Indexed 19632 HMMs (19632 names and 19632 accessions).
SSI index written to file Pfam-A.hmm.h3m.ssi


In [6]:
# сделали cntl+c, cntl+v столбца из таблицы в pfam_domains

with open('./data/pfam_domains', 'r') as file:
    pfam_domains_str = ' '.join(file.readlines())

for sep in ';,\n':
    pfam_domains_str = pfam_domains_str.replace(sep, ' ')

pfam_domains = set()
for word in pfam_domains_str.split():
    if word[:2] == 'PF':
        pfam_domains.add(word)


In [ ]:
!hmmsearch --tblout search.csv Pfam-A.hmm ./data/protein.faa

In [7]:
with open('search.csv', mode='r', newline='', encoding='utf-8') as file:
    data = file.readlines()

# оставляем интересные строчки
data = list(filter(lambda line : any(domain in line for domain in pfam_domains), data))


In [8]:
import pandas as pd

df = pd.DataFrame({'family': [], 'gene': []})

used_genes = set()

for line in data:
    elements = line.split()
    assert len(elements) >= 4
    if elements[2] in used_genes:
        continue
    used_genes.add(elements[2])
    df.loc[len(df.index)] = [elements[3],elements[2]]

df

,family,gene
0,PF00244.23,14-3-3
1,PF00389.33,2-Hacid_dh
2,PF02826.22,2-Hacid_dh_C
3,PF13532.9,2OG-FeII_Oxy_2
4,PF09038.13,53-BP1_Tudor
...,...,...
433,PF12660.10,zf-TFIIIC
434,PF02148.22,zf-UBP
435,PF02207.23,zf-UBR
436,PF18408.4,zf_Hakai


In [9]:
df.to_csv('./data/table.csv')

In [2]:
!pip install transformers
!pip install biopython

In [174]:
import torch
from torch import nn
import transformers
from transformers import BertTokenizer, BertForTokenClassification
import numpy as np
from Bio import SeqIO
from io import StringIO, BytesIO
from tqdm import tqdm
import pickle
import scipy
from scipy import ndimage

In [5]:
def seq2kmer(seq, k):
    kmer = [seq[x:x+k] for x in range(len(seq)+1-k)]
    return kmer

def split_seq(seq, length = 512, pad = 16):
    res = []
    for st in range(0, len(seq), length - pad):
        end = min(st+512, len(seq))
        res.append(seq[st:end])
    return res
        
def stitch_np_seq(np_seqs, pad = 16):
    res = np.array([])
    for seq in np_seqs:
        res = res[:-pad]
        res = np.concatenate([res,seq])
    return res

In [7]:
model = 'HG kouzine' #@param ["HG chipseq", "HG kouzine", "MM chipseq", "MM kouzine"]
model_confidence_threshold = 0.5 #@param {type:"number"}
minimum_sequence_length = 10 #@param {type:"integer"}

In [9]:
if model == 'HG chipseq':
    model_id = '1VAsp8I904y_J0PUhAQqpSlCn1IqfG0FB'
elif model == 'HG kouzine':
    model_id = '1dAeAt5Gu2cadwDhbc7OnenUgDLHlUvkx'
elif model == 'MM curax':
    model_id = '1W6GEgHNoitlB-xXJbLJ_jDW4BF35W1Sd'
elif model == 'MM kouzine':
    model_id = '1dXpQFmheClKXIEoqcZ7kgCwx6hzVCv3H'

In [ ]:
!gdown $model_id
!gdown 10sF8Ywktd96HqAL0CwvlZZUUGj05CGk5
!gdown 16bT7HDv71aRwyh3gBUbKwign1mtyLD2d
!gdown 1EE9goZ2JRSD8UTx501q71lGCk-CK3kqG
!gdown 1gZZdtAoDnDiLQqjQfGyuwt268Pe5sXW0 


In [ ]:
!mkdir 6-new-12w-0
!mv pytorch_model.bin 6-new-12w-0/
!mv config.json 6-new-12w-0/
!mv special_tokens_map.json 6-new-12w-0/
!mv tokenizer_config.json 6-new-12w-0/
!mv vocab.txt 6-new-12w-0/

In [3]:
torch.cuda.is_available()

True

In [11]:
tokenizer = BertTokenizer.from_pretrained('6-new-12w-0/')
model = BertForTokenClassification.from_pretrained('6-new-12w-0/')
model.cuda()

BertForTokenClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(4101, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSdpaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12,

In [33]:
key = './data/genomic.fna'
out = []
print(key)
out.append(key)
result_dict = {}
genome = None
with open('./data/genomic.fna', 'rb') as file:
    genome = file.read()

for seq_record in SeqIO.parse(StringIO(BytesIO(genome).read().decode('UTF-8')), 'fasta'):
    kmer_seq = seq2kmer(str(seq_record.seq).upper(), 6)
    seq_pieces = split_seq(kmer_seq)
    print(seq_record.name)
    out.append(seq_record.name)
    with torch.no_grad():
        preds = []
        for seq_piece in tqdm(seq_pieces):
            input_ids = torch.LongTensor(tokenizer.encode(' '.join(seq_piece), add_special_tokens=False))
            outputs = torch.softmax(model(input_ids.cuda().unsqueeze(0))[-1],axis = -1)[0,:,1]
            preds.append(outputs.cpu().numpy())
    result_dict[seq_record.name] = stitch_np_seq(preds)
    
   
    
    labeled, max_label = scipy.ndimage.label(result_dict[seq_record.name]>model_confidence_threshold)
    print('  start     end')
    out.append('  start     end')
    for label in range(1, max_label+1):
        candidate = np.where(labeled == label)[0]
        candidate_length = candidate.shape[0]
        if candidate_length>minimum_sequence_length:
            print('{:8}'.format(candidate[0]), '{:8}'.format(candidate[-1]))
            out.append('{:8}'.format(candidate[0]) + '{:8}'.format(candidate[-1]))
    
with open(key + '.preds.pkl',"wb") as fh:
    pickle.dump(result_dict, fh)
print()

with open('text_predictions.txt',"w") as fh:
    for item in out:
        fh.write("%s\n" % item)

./data/genomic.fna
NHQK01000001.1


  0%|          | 0/3528 [00:00<?, ?it/s]C:\Users\Stasana\anaconda3\Lib\site-packages\transformers\models\bert\modeling_bert.py:435: UserWarning: 1Torch was not compiled with flash attention. (Triggered internally at ..\aten\src\ATen\native\transformers\cuda\sdp_utils.cpp:455.)
  attn_output = torch.nn.functional.scaled_dot_product_attention(
100%|██████████| 3528/3528 [01:24<00:00, 41.70it/s]


  start     end
     429      446
     454      469
    1232     1248
    1260     1274
    2054     2071
    2080     2095
    2860     2877
    2886     2901
    3647     3659
    3669     3684
    4444     4462
    4473     4489
    5266     5280
    6042     6058
    6068     6083
    6844     6862
    6871     6886
    7652     7667
    8127     8142
    9081     9097
    9814     9833
   10190    10207
   12537    12555
   15806    15818
   17700    17715
   19829    19839
   20571    20590
   20858    20871
   21048    21059
   21758    21793
   21796    21815
   26006    26025
   26177    26191
   26693    26706
   27362    27373
   27784    27798
   28630    28648
   29184    29198
   29361    29376
   29878    29891
   31165    31186
   31988    32000
   33026    33040
   35437    35450
   37052    37066
   38633    38650
   38692    38705
   40648    40660
   41781    41795
   42261    42273
   43132    43145
   43975    43989
   44865    44877
   44884    44900
   45606    

100%|██████████| 2137/2137 [00:49<00:00, 43.48it/s]


  start     end
     219      233
    1984     1998
    3004     3018
    4393     4411
    5781     5795
    6039     6052
    7162     7176
    9771     9789
   12193    12209
   15561    15575
   15607    15624
   17468    17478
   18295    18308
   18888    18904
   24067    24083
   24509    24524
   27756    27772
   28674    28690
   31415    31425
   31670    31683
   32340    32358
   32373    32387
   32824    32837
   32914    32926
   33268    33281
   33286    33302
   34398    34417
   34448    34478
   34983    34996
   36209    36222
   37053    37076
   37864    37878
   37937    37949
   38070    38083
   38808    38825
   41661    41677
   43393    43410
   43476    43489
   43789    43806
   43877    43894
   44130    44143
   44212    44225
   44783    44798
   50996    51010
   54729    54741
   55189    55200
   55229    55250
   57361    57372
   57514    57527
   60119    60132
   66331    66347
   70830    70842
   72029    72041
   72090    72111
   72826    

...

  start     end
     280      294
     464      479
    1047     1060
    1257     1272
    1818     1832
    2000     2015
    2027     2044
    2582     2595
    2791     2807
    3352     3366
    3532     3548
    3560     3577
    4115     4128
    4296     4311
    4323     4340
    4878     4892
    5061     5075
    5087     5104
    6851     6861
    7396     7411
   13657    13671
   15981    16003
   16086    16099
   17087    17101
   20382    20394
   20601    20614
   21064    21079
   22063    22079
   22275    22291
   22749    22770
   24936    24947
   28675    28687
   30640    30655
   32569    32585
   33080    33095
   33153    33166
   33287    33301
   33847    33859
   34014    34030
   37493    37510
   39208    39229
   39304    39319
   39695    39712
   39947    39961
   42158    42173
   42788    42799
   44595    44610
   45775    45789
   47690    47706
   49339    49352
   49894    49910
   49995    50009
   50274    50290
   52703    52717
   58742    

100%|██████████| 400/400 [00:09<00:00, 43.03it/s]


  start     end
      29       44
     798      813
    1337     1353
    1800     1814
    2021     2034
    2208     2220
    2504     2522
    3194     3205
    3979     4000
    4341     4355
    6275     6288
    6336     6351
    6722     6737
    6911     6932
    7196     7206
    7317     7334
    7655     7670
    7776     7793
    7989     8005
    8195     8210
    9054     9065
    9236     9263
    9635     9645
   10766    10783
   10833    10850
   11256    11271
   12926    12940
   15991    16004
   18133    18146
   18439    18453
   20432    20449
   20713    20732
   24656    24667
   24845    24875
   25533    25544
   26686    26698
   26902    26919
   29343    29354
   36485    36497
   36569    36581
   38243    38255
   38499    38519
   38540    38556
   39038    39052
   40353    40366
   40728    40742
   41717    41734
   41976    41992
   43079    43092
   43450    43462
   43648    43671
   44364    44378
   46360    46372
   47067    47088
   47269    

100%|██████████| 389/389 [00:09<00:00, 42.72it/s]


  start     end
   10161    10172
   11348    11359
   11855    11871
   11949    11967
   12017    12031
   12136    12148
   12982    12998
   13013    13029
   13055    13069
   13155    13170
   13208    13223
   13253    13269
   13318    13334
   13565    13576
   14531    14547
   15360    15375
   15805    15818
   16832    16847
   17068    17085
   17780    17798
   17926    17939
   18368    18385
   21282    21295
   21334    21352
   22052    22062
   23057    23076
   25834    25848
   26465    26483
   26514    26527
   26649    26664
   27728    27742
   27954    27968
   36005    36022
   37995    38011
   39600    39618
   40492    40508
   40834    40848
   40925    40945
   41276    41292
   41650    41663
   41752    41767
   42823    42836
   51512    51527
   52153    52166
   53543    53557
   56023    56037
   56227    56239
   56813    56828
   57570    57584
   58041    58056
   58601    58613
   59877    59894
   60381    60394
   60974    60988
   60997    

В тетрадке с примером из прошлого года собирается так: gcc zhunt3-alan.c -lm -o zhunt3
по умолчанию gcc собирает debug версию, которая в разы медленнее. Если добавить флаг -O2 время работы уменьшится в разы

In [39]:
!gcc zhunt3-alan.c -lm -o -O2 zhunt3

zhunt3-alan.c: In function 'calculate_zscore':
zhunt3-alan.c:566:9: warning: passing argument 1 of 'time' from incompatible pointer type [-Wincompatible-pointer-types]
  566 |   time( &begintime );
      |         ^~~~~~~~~~
      |         |
      |         long int *
In file included from zhunt3-alan.c:30:
C:/msys64/ucrt64/include/time.h:258:47: note: expected 'time_t *' {aka 'long long int *'} but argument is of type 'long int *'
  258 | static __inline time_t __CRTDECL time(time_t *_Time) { return _time64(_Time); }
      |                                       ~~~~~~~~^~~~~
zhunt3-alan.c:599:9: warning: passing argument 1 of 'time' from incompatible pointer type [-Wincompatible-pointer-types]
  599 |   time( &endtime );
      |         ^~~~~~~~
      |         |
      |         long int *
C:/msys64/ucrt64/include/time.h:258:47: note: expected 'time_t *' {aka 'long long int *'} but argument is of type 'long int *'
  258 | static __inline time_t __CRTDECL time(time_t *_Time) { return

In [49]:
!zhunt3 12 8 12 ./data/genomic.fna

dinucleotides 12
min/max 8 12
min/max 8 12
operating on ./data/genomic.fna
calculating zscore
opening ./data/genomic.fna
inputting sequence
opening ./data/genomic.fna.Z-SCORE

 run time=1718616240 sec
use min/max 8 12
analyzing_zscore
opening ./data/genomic.fna.Z-SCORE
opening ./data/genomic.fna
inputting sequence


In [64]:
import re
import pandas as pd
from Bio import SeqIO

df = pd.DataFrame(columns=['Chromosome', 'Start', 'End', 'Sequence', 'Strand'])
pattern = '(G{3,5}[ATGC]{1,7}){3,}G{3,5}'

def foo(strand):
    for record in SeqIO.parse("./data/genomic.fna",'fasta'):
        (name, sequence) = (record.id, str(record.seq))
        for m in re.finditer(pattern, str(record.seq),re.IGNORECASE):
            df.loc[len(df.index)] = [name, m.start(), m.end(), m.group(0), strand]

foo('+')

pair = {'G': 'C', 'C': 'G', 'A': 'T', 'T': 'A'}
pattern = ''.join([(pair[i] if i in pair else i) for i in pattern])

foo('-')


In [72]:
from fuc import pybed

bf = pybed.BedFrame.from_frame(meta=[], data=df)
bf.to_file('quadr.bed')

ModuleNotFoundError: No module named 'fuc'

In [78]:
annotation_df = pd.read_csv('./data/genomic.gff', sep='\t', comment='#', header=None, names=[
    "seqid", "source", "type", "start", "end", "score", "strand", "phase", "attributes"
])

In [108]:
annotation_df.head(5)

,seqid,source,type,start,end,score,strand,phase,attributes
0,NHQK01000001.1,Genbank,region,1,1749730,.,+,.,ID=NHQK01000001.1:1..1749730;Dbxref=taxon:2992...
1,NHQK01000001.1,Genbank,gene,9880,11571,.,+,.,ID=gene-Pcac1_g1;Name=Pcac1_g1;end_range=11571...
2,NHQK01000001.1,Genbank,mRNA,9880,11571,.,+,.,ID=rna-gnl|WGS:NHQK|Pcac1_g1.t1;Parent=gene-Pc...
3,NHQK01000001.1,Genbank,exon,9880,11571,.,+,.,ID=exon-gnl|WGS:NHQK|Pcac1_g1.t1-1;Parent=rna-...
4,NHQK01000001.1,Genbank,CDS,9880,11571,.,+,0,ID=cds-KAG2790607.1;Parent=rna-gnl|WGS:NHQK|Pc...


In [146]:
genes = annotation_df[annotation_df['type'] == 'gene']
print(len(genes))

29544


In [144]:
def get_gene_exons(gene):
    is_exon = annotation_df['type'] == 'exon'
    is_this_gene = (annotation_df['seqid'] == gene.seqid) & (annotation_df['start'] >= gene.start) & (annotation_df['end'] <= gene.end)
    return annotation_df[is_exon & is_this_gene]

intrones = pd.DataFrame(columns=["seqid", "source", "type", "start", "end", "score", "strand", "phase", "attributes"])

genes = annotation_df[annotation_df['type'] == 'gene']
i = 0
for gene in genes.itertuples():
    exons = get_gene_exons(gene).sort_values(by='start')
    if i % 100 == 0:
        print(i)
    i += 1
    previous_exon_end = None
    for exon in exons.itertuples():
        if previous_exon_end is not None:
            intron_start = previous_exon_end + 1
            intron_end = exon.start - 1
            if intron_start < intron_end:
                intrones.loc[len(intones.index)] = [gene.seqid, "predicted", "intron", intron_start, intron_end, '.', gene.strand, '.', gene.attributes]
        previous_exon_end = exon.end

0
100
200
300
400
500
600
700
800
900
1000
1100
1200
1300
1400
1500
1600
1700
1800
1900
2000
2100
2200
2300
2400
2500
2600
2700
2800
2900
3000
3100
3200
3300
3400
3500
3600
3700
3800
3900
4000
4100
4200
4300
4400
4500
4600
4700
4800
4900
5000
5100
5200
5300
5400
5500
5600
5700
5800
5900
6000
6100
6200
6300
6400
6500
6600
6700
6800
6900
7000
7100
7200
7300
7400
7500
7600
7700
7800
7900
8000
8100
8200
8300
8400
8500
8600
8700
8800
8900
9000
9100
9200
9300
9400
9500
9600
9700
9800
9900
10000
10100
10200
10300
10400
10500
10600
10700
10800
10900
11000
11100
11200
11300
11400
11500
11600
11700
11800
11900
12000
12100
12200
12300
12400
12500
12600
12700
12800
12900
13000
13100
13200
13300
13400
13500
13600
13700
13800
13900
14000
14100
14200
14300
14400
14500
14600
14700
14800
14900
15000
15100
15200
15300
15400
15500
15600
15700
15800
15900
16000
16100
16200
16300
16400
16500
16600
16700
16800
16900
17000
17100
17200
17300
17400
17500
17600
17700
17800
17900
18000
18100
18200
18300
18400
18

In [166]:
df = pd.concat([annotation_df, intrones], ignore_index=True)
df = df[df['type'].isin(['gene', 'exon', 'intron'])]
df

,seqid,source,type,start,end,score,strand,phase,attributes
1,NHQK01000001.1,Genbank,gene,9880,11571,.,+,.,ID=gene-Pcac1_g1;Name=Pcac1_g1;end_range=11571...
3,NHQK01000001.1,Genbank,exon,9880,11571,.,+,.,ID=exon-gnl|WGS:NHQK|Pcac1_g1.t1-1;Parent=rna-...
5,NHQK01000001.1,Genbank,gene,12460,13335,.,-,.,ID=gene-Pcac1_g2;Name=Pcac1_g2;end_range=13335...
7,NHQK01000001.1,Genbank,exon,12460,13335,.,-,.,ID=exon-gnl|WGS:NHQK|Pcac1_g2.t1-1;Parent=rna-...
9,NHQK01000001.1,Genbank,gene,15589,17761,.,-,.,ID=gene-Pcac1_g3;Name=Pcac1_g3;end_range=17761...
...,...,...,...,...,...,...,...,...,...
183512,NHQK01000115.1,predicted,intron,20403,20468,.,+,.,ID=gene-Pcac1_g27655;Name=Pcac1_g27655;end_ran...
183513,NHQK01000115.1,predicted,intron,24068,24143,.,+,.,ID=gene-Pcac1_g27656;Name=Pcac1_g27656;end_ran...
183514,NHQK01000115.1,predicted,intron,24335,24404,.,+,.,ID=gene-Pcac1_g27656;Name=Pcac1_g27656;end_ran...
183515,NHQK01000115.1,predicted,intron,25245,25318,.,+,.,ID=gene-Pcac1_g27656;Name=Pcac1_g27656;end_ran...


In [168]:
def classify(structure):
    seqid = structure['seqid']
    start = structure['start']
    end = structure['end']
    length = end - start + 1
    midpoint = start + length // 2

    annon_slice = df[
        (df['seqid'] == seqid) &
        (df['start'] - 1000 <= end) &
        (df['end'] + 1000 >= start)
    ]

    for _, row in annon_slice.iterrows():
        if row['type'] == 'gene':
            tss = row['start']
            tes = row['end']
            if row['strand'] == '-':
                (tss, tes) = (tes, tss)

            if midpoint in range(tss - (1000 if row['strand'] == '+' else 0), tss + 1 + (1000 if row['strand'] == '-' else 0)):
                return "promoter"

            if midpoint in range(tes - (200 if row['strand'] == '-' else 0), tes + 1 + (200 if row['strand'] == '+' else 0)):
                return "downstream"
        else:
          if row['start'] <= midpoint <= row['end']:
              return row['type']

    return "intergenic"

In [178]:
pattern = r"((?:G{3,}[ATGC]{1,7}){3,}G{3,5})|((?:C{3,}[ATGC]{1,7}){3,}C{3,5})"

genome_records = list(SeqIO.parse("./data/genomic.fna", "fasta"))

quad_df = pd.DataFrame(columns=['seqid', 'start', 'end', 'sequence'])

for record in genome_records:
    seqid = record.id
    sequence = str(record.seq)

    for match in re.finditer(pattern, sequence, re.IGNORECASE):
        match_seq = match.group(0)
        quad_df.loc[len(quad_df.index)] = [seqid, match.start() + 1, match.end(), match_seq]

quad_df


,seqid,start,end,sequence
0,NHQK01000001.1,79699,79716,cccccccccccccccccc
1,NHQK01000001.1,93450,93467,CCCTCCCGATCCCACCCC
2,NHQK01000001.1,103415,103437,gggggggggggggggtaacgggg
3,NHQK01000001.1,109079,109114,CCCTGCTGGCccgaaccccccccccccccccccccc
4,NHQK01000001.1,112442,112460,CCCACCCTGAACCCCACCC
...,...,...,...,...
1895,NHQK01000098.1,173710,173724,ccccccccccccccc
1896,NHQK01000098.1,190730,190751,ccccgttacccccccccccccc
1897,NHQK01000099.1,4284,4316,GGGGGTGAGGGACGACGTGGGCACTGGGGTGGG
1898,NHQK01000099.1,81649,81663,cccacccacccaccC


In [198]:
result = [classify(quad._asdict()) for quad in quad_df.itertuples()]
set(result)

{'downstream', 'exon', 'intergenic', 'intron', 'promoter'}

In [200]:
np.unique(result, return_counts=True)

(array(['downstream', 'exon', 'intergenic', 'intron', 'promoter'],
       dtype='<U10'),
 array([ 93, 187, 777,  15, 828], dtype=int64))

In [202]:
np.unique(result, return_counts=True)[1] / len(result) * 100

array([ 4.89473684,  9.84210526, 40.89473684,  0.78947368, 43.57894737])

In [206]:
quad_df.to_csv('./data/quad.bed', sep='\t', header=False, index=False, columns=('seqid', 'start', 'end'))

надо было в коде zhunt фильтровать... ну ладно, 5 гигов данных

In [225]:
chunks = pd.read_csv("./data/genomic.fna.Z-SCORE",
              skiprows=1,
              names=["start","end","3","4","5","score","seq","8"],
              delim_whitespace=True,
              chunksize=10000)

filtered_chunks = []

i = 0
for chunk in chunks:
    if i % 100 == 0:
        print(i)
    i += 1
    filtered_chunk = chunk[chunk['score'] > 4000]
    filtered_chunks.append(filtered_chunk)

zhunt_df = pd.concat(filtered_chunks, ignore_index=True)


0
100
200
300
400
500
600
700
800
900
1000
1100
1200
1300
1400
1500
1600
1700
1800
1900
2000
2100
2200
2300
2400
2500
2600
2700
2800
2900
3000
3100
3200
3300
3400
3500
3600
3700
3800
3900
4000
4100
4200
4300
4400
4500
4600
4700
4800
4900
5000
5100
5200
5300
5400
5500
5600
5700
5800
5900
6000
6100
6200
6300
6400
6500


In [227]:
zhunt_df

,start,end,3,4,5,score,seq,8
0,9836,9852,16,20.167,34.143,4105.351,accccagcgcgcgcgg,SASASASASASASASA
1,10210,10226,16,20.122,32.524,4368.947,gacgttcccgcgcgcg,ASASASASASASASAS
2,10211,10227,16,19.960,36.208,5475.943,acgttcccgcgcgcgc,SASASASASASASASA
3,10212,10228,16,20.049,33.086,4831.423,cgttcccgcgcgcgct,ASASASASASASASAS
4,10213,10229,16,20.067,33.312,4718.097,gttcccgcgcgcgctt,SASASASASASASASA
...,...,...,...,...,...,...,...,...
50211,65948777,65948793,16,19.510,43.923,10410.390,catacgcgcgtgcagg,ASASASASASASASAS
50212,65948778,65948794,16,19.940,41.064,5632.859,atacgcgcgtgcaggt,SASASASASASASASA
50213,65948779,65948795,16,19.849,39.975,6403.388,tacgcgcgtgcaggtc,ASASASASASASASAS
50214,65948780,65948796,16,20.134,39.290,4299.587,acgcgcgtgcaggtcg,SASASASASASASAAS


In [237]:
seq_df = annotation_df[annotation_df['type'] == 'region']
seq_df = seq_df.reset_index(drop=True)
seq_df['cumulative_end'] = seq_df['end'].cumsum()
seq_df

,seqid,source,type,start,end,score,strand,phase,attributes,cumulative_end
0,NHQK01000001.1,Genbank,region,1,1749730,.,+,.,ID=NHQK01000001.1:1..1749730;Dbxref=taxon:2992...,1749730
1,NHQK01000010.1,Genbank,region,1,1059835,.,+,.,ID=NHQK01000010.1:1..1059835;Dbxref=taxon:2992...,2809565
2,NHQK01000100.1,Genbank,region,1,191713,.,+,.,ID=NHQK01000100.1:1..191713;Dbxref=taxon:29920...,3001278
3,NHQK01000101.1,Genbank,region,1,183340,.,+,.,ID=NHQK01000101.1:1..183340;Dbxref=taxon:29920...,3184618
4,NHQK01000102.1,Genbank,region,1,173919,.,+,.,ID=NHQK01000102.1:1..173919;Dbxref=taxon:29920...,3358537
...,...,...,...,...,...,...,...,...,...,...
189,NHQK01000095.1,Genbank,region,1,214371,.,+,.,ID=NHQK01000095.1:1..214371;Dbxref=taxon:29920...,65144651
190,NHQK01000096.1,Genbank,region,1,212795,.,+,.,ID=NHQK01000096.1:1..212795;Dbxref=taxon:29920...,65357446
191,NHQK01000097.1,Genbank,region,1,211485,.,+,.,ID=NHQK01000097.1:1..211485;Dbxref=taxon:29920...,65568931
192,NHQK01000098.1,Genbank,region,1,198018,.,+,.,ID=NHQK01000098.1:1..198018;Dbxref=taxon:29920...,65766949


In [241]:
records = pd.DataFrame(columns=['seqid', 'start', 'end', 'seq'])

for index, row in zhunt_df.iterrows():
    for seq_index, seq_row in seq_df.iterrows():
        if row['start'] <= seq_row['cumulative_end']:
            seqid = seq_row['seqid']
            prev_cumulative_end = seq_df.iloc[seq_index - 1]['cumulative_end'] if seq_index > 0 else 0
            corrected_start = row['start'] - prev_cumulative_end
            corrected_end = row['end'] - prev_cumulative_end
            
            records.loc[len(records.index)] = [seqid, corrected_start, corrected_end, row['seq']]
            break

In [258]:
result = [classify(zhunt._asdict()) for zhunt in records.itertuples()]
set(result)

{'downstream', 'exon', 'intergenic', 'intron', 'promoter'}

In [260]:
np.unique(result, return_counts=True)

(array(['downstream', 'exon', 'intergenic', 'intron', 'promoter'],
       dtype='<U10'),
 array([ 2867, 21711,  9551,   465, 15622], dtype=int64))

In [262]:
np.unique(result, return_counts=True)[1]/(len(result))*100

array([ 5.70933567, 43.23522383, 19.01983432,  0.92599968, 31.1096065 ])

In [264]:
records.to_csv('./data/zhunt.bed', sep='\t', header=False, index=False, columns=('seqid', 'start', 'end'))

In [246]:
with open('./data/text_predictions.txt', 'r') as file:
    lines = file.readlines()

lines = lines[1:]

bert_df = pd.DataFrame(columns=['seqid', 'start', 'end'])

current_seqid = None

for line in lines:
    line = line.strip()
    if len(line.split()) == 1:
        current_seqid = line
    elif line and not line.startswith('start'):
        start, end = map(int, line.split())
        bert_df.loc[len(bert_df.index)] = [current_seqid, start, end]


In [248]:
bert_df

,seqid,start,end
0,NHQK01000001.1,429,446
1,NHQK01000001.1,454,469
2,NHQK01000001.1,1232,1248
3,NHQK01000001.1,1260,1274
4,NHQK01000001.1,2054,2071
...,...,...,...
54828,NHQK01000099.1,190859,190875
54829,NHQK01000099.1,191075,191091
54830,NHQK01000099.1,192058,192075
54831,NHQK01000099.1,192271,192287


In [250]:
result = [classify(bert._asdict()) for bert in bert_df.itertuples()]
set(result)

{'downstream', 'exon', 'intergenic', 'intron', 'promoter'}

In [252]:
np.unique(result, return_counts=True)

(array(['downstream', 'exon', 'intergenic', 'intron', 'promoter'],
       dtype='<U10'),
 array([ 2475, 29043,  7446,   424, 15445], dtype=int64))

In [254]:
np.unique(result, return_counts=True)[1]/(len(result))*100

array([ 4.51370525, 52.96627943, 13.57941386,  0.77325698, 28.16734448])

In [256]:
bert_df.to_csv('./data/bert.bed', sep='\t', header=False, index=False, columns=('seqid', 'start', 'end'))